In [1]:
!pip install tensorflow numpy pandas scikit-learn tabulate

In [2]:
from google.colab import files
import os

In [3]:
os.makedirs('./data', exist_ok=True)

In [4]:
print("Uploading the following CSV files:")
print("- content_item_train.csv")
print("- content_user_train.csv")
print("- content_y_train.csv")
print("- content_item_train_header.txt")
print("- content_user_train_header.txt")
print("- content_item_vecs.csv")
print("- content_movie_list.csv")
print("- content_user_to_genre.pickle")

Uploading the following CSV files:
- content_item_train.csv
- content_user_train.csv
- content_y_train.csv
- content_item_train_header.txt
- content_user_train_header.txt
- content_item_vecs.csv
- content_movie_list.csv
- content_user_to_genre.pickle


In [5]:
uploaded = files.upload()

Saving content_item_train.csv to content_item_train.csv
Saving content_item_train_header.txt to content_item_train_header.txt
Saving content_item_vecs.csv to content_item_vecs.csv
Saving content_movie_list.csv to content_movie_list.csv
Saving content_user_to_genre.pickle to content_user_to_genre.pickle
Saving content_user_train.csv to content_user_train.csv
Saving content_user_train_header.txt to content_user_train_header.txt
Saving content_y_train.csv to content_y_train.csv


In [6]:
# Move uploaded files to data directory
for filename in uploaded.keys():
    os.rename(filename, f'./data/{filename}')

In [7]:
# Cell 3: Import libraries and utility functions
import numpy as np
import numpy.ma as ma
from numpy import genfromtxt
from collections import defaultdict
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pickle
import csv

In [8]:
pd.set_option("display.precision", 1)

In [9]:
# Cell 4: Define load_data function
def load_data():
    item_train = genfromtxt('./data/content_item_train.csv', delimiter=',')
    user_train = genfromtxt('./data/content_user_train.csv', delimiter=',')
    y_train = genfromtxt('./data/content_y_train.csv', delimiter=',')

    with open('./data/content_item_train_header.txt', newline='') as f:
        item_features = list(csv.reader(f))[0]
    with open('./data/content_user_train_header.txt', newline='') as f:
        user_features = list(csv.reader(f))[0]

    item_vecs = genfromtxt('./data/content_item_vecs.csv', delimiter=',')

    movie_dict = defaultdict(dict)
    count = 0
    with open('./data/content_movie_list.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for line in reader:
            if count == 0:
                count += 1
            else:
                count += 1
                movie_id = int(line[0])
                movie_dict[movie_id]["title"] = line[1]
                movie_dict[movie_id]["genres"] = line[2]

    with open('./data/content_user_to_genre.pickle', 'rb') as f:
        user_to_genre = pickle.load(f)

    return (item_train, user_train, y_train, item_features,
            user_features, item_vecs, movie_dict, user_to_genre)

In [10]:
# Cell 5: Load data and set configuration
item_train, user_train, y_train, item_features, user_features, \
item_vecs, movie_dict, user_to_genre = load_data()

num_user_features = user_train.shape[1] - 3
num_item_features = item_train.shape[1] - 1
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items
scaledata = True

print(f"Number of training vectors: {len(item_train)}")
print(f"Number of user features: {num_user_features}")
print(f"Number of item features: {num_item_features}")
print(f"User features: {user_features}")
print(f"Item features: {item_features}")

Number of training vectors: 58187
Number of user features: 14
Number of item features: 16
User features: ['user id', 'rating count', 'rating ave', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']
Item features: ['movie id', 'year', 'ave rating', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']


/tmp/ipython-input-831037181.py:28: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  user_to_genre = pickle.load(f)


In [11]:
# Cell 6: Scale training data
if scaledata:
    item_train_save = item_train
    user_train_save = user_train

    scalerItem = StandardScaler()
    scalerItem.fit(item_train)
    item_train = scalerItem.transform(item_train)

    scalerUser = StandardScaler()
    scalerUser.fit(user_train)
    user_train = scalerUser.transform(user_train)

    print("Item scaler check:", np.allclose(item_train_save,
                                            scalerItem.inverse_transform(item_train)))
    print("User scaler check:", np.allclose(user_train_save,
                                            scalerUser.inverse_transform(user_train)))

Item scaler check: True
User scaler check: True


In [12]:
# Cell 7: Train-test split
item_train, item_test = train_test_split(item_train, train_size=0.80,
                                         shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80,
                                         shuffle=True, random_state=1)
y_train, y_test = train_test_split(y_train, train_size=0.80,
                                   shuffle=True, random_state=1)

print(f"Movie/item training data shape: {item_train.shape}")
print(f"Movie/item test data shape: {item_test.shape}")

Movie/item training data shape: (46549, 17)
Movie/item test data shape: (11638, 17)


In [13]:
# Cell 8: Scale target variable
scaler = MinMaxScaler((-1, 1))
scaler.fit(y_train.reshape(-1, 1))
ynorm_train = scaler.transform(y_train.reshape(-1, 1))
ynorm_test = scaler.transform(y_test.reshape(-1, 1))
print(f"Normalized y shapes: {ynorm_train.shape}, {ynorm_test.shape}")

Normalized y shapes: (46549, 1), (11638, 1)


In [14]:
# Cell 9: Build neural network model
num_outputs = 32
tf.random.set_seed(1)

user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs, activation='linear'),
])

# Create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = tf.keras.layers.UnitNormalization(axis=1)(vu)

# Create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = tf.keras.layers.UnitNormalization(axis=1)(vm)

# Compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# Specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 14)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 16)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 32)        │     40,864 │ input_layer[0][0] │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_1        │ (None, 32)        │     41,376 │ input_layer_2[0]… │
│ (Sequential)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ unit_normalization  │ (None, 32)        │          0 │ sequential[0][0]  │
│ (UnitNormalization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ unit_normalization… │ (None, 32)        │          0 │ sequential_1[0][… │
│ (UnitNormalization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 1)         │          0 │ unit_normalizati… │
│                     │                   │            │ unit_normalizati… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 82,240 (321.25 KB)

 Trainable params: 82,240 (321.25 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Cell 10: Compile model
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt, loss=cost_fn)

In [16]:
# Cell 11: Train model
print("Starting training...")
history = model.fit([user_train[:, u_s:], item_train[:, i_s:]],
                   ynorm_train, epochs=30, validation_split=0.1, verbose=1)

Starting training...
Epoch 1/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - loss: 0.1343 - val_loss: 0.1206
Epoch 2/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.1207 - val_loss: 0.1169
Epoch 3/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1184 - val_loss: 0.1152
Epoch 4/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1171 - val_loss: 0.1135
Epoch 5/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1160 - val_loss: 0.1127
Epoch 6/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1149 - val_loss: 0.1119
Epoch 7/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1137 - val_loss: 0.1108
Epoch 8/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1126 - val_loss: 0.1100
Epoch 9/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1116 - val_loss: 0.1095
Epoch 10/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1107 - val_loss: 0.1093
Epoch 11/30
1310/1310 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.1098 - val_loss: 0.1089
Epoch 12/

In [17]:
# Cell 12: Evaluate model
test_loss = model.evaluate([user_test[:, u_s:], item_test[:, i_s:]], ynorm_test)
print(f"Test Loss: {test_loss}")

364/364 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.1070
Test Loss: 0.10643234103918076


In [18]:
# Cell 13: Create movie feature model
input_item_m = tf.keras.layers.Input(shape=(num_item_features,))
vm_m = item_NN(input_item_m)
vm_m = tf.keras.layers.UnitNormalization(axis=1)(vm_m)
model_m = tf.keras.Model(input_item_m, vm_m)
model_m.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 32)             │        41,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ unit_normalization_2            │ (None, 32)             │             0 │
│ (UnitNormalization)             │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 41,376 (161.62 KB)

 Trainable params: 41,376 (161.62 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Generate movie feature vectors
scaled_item_vecs = scalerItem.transform(item_vecs)
vms = model_m.predict(scaled_item_vecs[:, i_s:])
print(f"Size of all predicted movie feature vectors: {vms.shape}")

59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Size of all predicted movie feature vectors: (1883, 32)


In [20]:
# Cell 14: Save all models and artifacts
os.makedirs('./models', exist_ok=True)

# Save the main model
model.save('./models/recommendation_model.h5')
print("Main model saved")

# Save user and item neural networks separately
user_NN.save('./models/user_nn.h5')
item_NN.save('./models/item_nn.h5')
print("User and Item NNs saved")

# Save scalers
import joblib
joblib.dump(scalerUser, './models/scaler_user.pkl')
joblib.dump(scalerItem, './models/scaler_item.pkl')
joblib.dump(scaler, './models/scaler_y.pkl')
print("Scalers saved")

# Save configuration
config = {
    'num_user_features': num_user_features,
    'num_item_features': num_item_features,
    'uvs': uvs,
    'ivs': ivs,
    'u_s': u_s,
    'i_s': i_s,
    'num_outputs': num_outputs,
    'user_features': user_features,
    'item_features': item_features
}
with open('./models/config.pkl', 'wb') as f:
    pickle.dump(config, f)
print("Configuration saved")

# Save item vectors and movie feature vectors
np.save('./models/item_vecs.npy', item_vecs)
np.save('./models/movie_feature_vectors.npy', vms)
print("Item vectors and movie features saved")

# Save movie dictionary
with open('./models/movie_dict.pkl', 'wb') as f:
    pickle.dump(movie_dict, f)
print("Movie dictionary saved")

Main model saved
User and Item NNs saved
Scalers saved
Configuration saved
Item vectors and movie features saved
Movie dictionary saved


In [21]:
# Cell 15: Download all model files
from google.colab import files
import shutil

# Create a zip file of all models
shutil.make_archive('recommendation_models', 'zip', './models')
files.download('recommendation_models.zip')

print("\n✅ Training complete!")
print("📦 Download 'recommendation_models.zip' and extract it for FastAPI deployment")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Training complete!
📦 Download 'recommendation_models.zip' and extract it for FastAPI deployment
